# Coursera Capstone Project

In [81]:
from IPython.display import Image
img = "https://wallpapercave.com/wp/NHNWfIX.jpg"
Image(url=img)

### 1. Import Necessary Libraries

In [82]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geocoder
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [83]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Athens").text

In [84]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [85]:
# create a list to store neighborhood data
neighborhoodList = []

In [86]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [87]:
# create a new DataFrame from the list
athens_df = pd.DataFrame({"Neighborhood": neighborhoodList})

athens_df.head()

,Neighborhood
0,"Aerides, Athens"
1,"Agios Eleftherios, Athens"
2,"Agios Panteleimonas, Athens"
3,Akadimia Platonos
4,"Akadimia, Athens"


In [88]:
# print the number of rows of the dataframe
athens_df.shape

(63, 1)

### 3. Get the geographical coordinates of Athens

In [89]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Athens, Greece'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [90]:
# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [get_latlng(neighborhood) for neighborhood in athens_df["Neighborhood"].tolist()]
coordinates

[[37.97614000000004, 23.73640000000006],
 [38.01997000000006, 23.726270000000056],
 [37.99672000000004, 23.727410000000077],
 [37.990950000000055, 23.70718000000005],
 [37.97753256017709, 23.73759171802083],
 [37.99106000000006, 23.764290000000074],
 [37.97614000000004, 23.73640000000006],
 [37.96948000000003, 23.70897000000008],
 [37.974469937821105, 23.719200000000058],
 [37.9316200676185, 23.721150975535412],
 [37.98645000000005, 23.741070000000036],
 [37.96667000000008, 23.669400000000053],
 [37.99793000000005, 23.775160000000028],
 [37.99268103211144, 23.767639433993164],
 [37.977150047752296, 23.747729973310978],
 [37.988170000000025, 23.733610000000056],
 [37.97872000000007, 23.714260000000024],
 [37.97614000000004, 23.73640000000006],
 [37.982920000000036, 23.770980000000066],
 [37.96002995614295, 23.740770069392383],
 [38.0204746539529, 23.845249735730313],
 [37.976410000000044, 23.755570000000034],
 [37.96691000000004, 23.74022000000008],
 [37.96691000000004, 23.7402200000000

In [91]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [92]:
# merge the coordinates into the original dataframe
athens_df['Latitude'] = df_coordinates['Latitude']
athens_df['Longitude'] = df_coordinates['Longitude']

In [93]:
# check the neighborhoods and the coordinates
print(athens_df.shape)
athens_df

(63, 3)


,Neighborhood,Latitude,Longitude
0,"Aerides, Athens",37.976140,23.736400
1,"Agios Eleftherios, Athens",38.019970,23.726270
2,"Agios Panteleimonas, Athens",37.996720,23.727410
3,Akadimia Platonos,37.990950,23.707180
4,"Akadimia, Athens",37.977533,23.737592
5,"Ampelokipoi, Athens",37.991060,23.764290
6,Anafiotika,37.976140,23.736400
7,Ano Petralona,37.969480,23.708970
8,Asteroskopeio,37.974470,23.719200
9,"Asyrmatos, Athens",37.931620,23.721151


In [94]:
# save the DataFrame as CSV file
athens_df.to_csv("athens_df.csv", index=False)

### 4. Create a map of Athens with neighborhoods superimposed on top

In [95]:
# get the coordinates of Athens
address = 'Athens, Greece'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Athens, Greece {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Athens, Greece 37.9841493, 23.7279843.


In [96]:
# create map of Toronto using latitude and longitude values
athens_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(athens_df['Latitude'], athens_df['Longitude'], athens_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(athens_map)  
    
athens_map

In [97]:
# save the map as HTML file
athens_map.save('map_kl.html')

### 5. Explore the neighborhoods using the Foursquare API

In [98]:
# define Foursquare Credentials and Version
CLIENT_ID = 'MFD11Q1KDZIHAXZPNBNSJHXWIXFJREXEBARIWDUTTG5MLY24' # your Foursquare ID
CLIENT_SECRET = 'TITPEFVTOU4KEHZKMM0EXECWD31GOH4VH2XMB3MZLHVPZWGX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MFD11Q1KDZIHAXZPNBNSJHXWIXFJREXEBARIWDUTTG5MLY24
CLIENT_SECRET:TITPEFVTOU4KEHZKMM0EXECWD31GOH4VH2XMB3MZLHVPZWGX


#### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [99]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(athens_df['Latitude'], athens_df['Longitude'], athens_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [100]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6296, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Aerides, Athens",37.97614,23.7364,Hotel Grande Bretagne,37.976254,23.735364,Hotel
1,"Aerides, Athens",37.97614,23.7364,Louis Vuitton,37.977704,23.736446,Boutique
2,"Aerides, Athens",37.97614,23.7364,Toyroom Athens,37.977593,23.735721,Nightclub
3,"Aerides, Athens",37.97614,23.7364,GB Roof Garden Restaurant,37.976116,23.735301,Roof Deck
4,"Aerides, Athens",37.97614,23.7364,Carpo,37.977183,23.738547,Gourmet Shop


#### Let's check how many venues were returned for each neighorhood

In [101]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Aerides, Athens",100,100,100,100,100,100
"Agios Eleftherios, Athens",100,100,100,100,100,100
"Agios Panteleimonas, Athens",100,100,100,100,100,100
Akadimia Platonos,100,100,100,100,100,100
"Akadimia, Athens",100,100,100,100,100,100
"Ampelokipoi, Athens",100,100,100,100,100,100
Anafiotika,100,100,100,100,100,100
Ano Petralona,100,100,100,100,100,100
Asteroskopeio,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [102]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 209 uniques categories.


In [103]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Hotel', 'Boutique', 'Nightclub', 'Roof Deck', 'Gourmet Shop',
       'Theater', 'Park', 'Coffee Shop', 'Bakery', 'Café',
       'Jewelry Store', 'Gym', 'Bookstore', 'Ice Cream Shop', 'Bar',
       'Sushi Restaurant', 'Bistro', 'Pie Shop', 'Museum', 'Dessert Shop',
       'Tea Room', 'Bagel Shop', 'Cocktail Bar',
       'Vegetarian / Vegan Restaurant', 'Sandwich Place', 'Wine Shop',
       'Wine Bar', 'History Museum', 'American Restaurant',
       'Chocolate Shop', 'Indie Movie Theater', 'Fast Food Restaurant',
       'Cupcake Shop', 'Garden', 'Furniture / Home Store', 'Whisky Bar',
       'Movie Theater', 'Plaza', 'Gift Shop', 'Burger Joint',
       'Middle Eastern Restaurant', 'Cheese Shop', 'Distillery',
       'Falafel Restaurant', 'Track Stadium', 'Tennis Court',
       'Historic Site', 'Bubble Tea Shop', 'Greek Restaurant', 'Kafenio'],
      dtype=object)

In [104]:
# check if the results contain "Sushi Restaurant"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

#### 6. Analyze Each Neighborhood

In [105]:
# one hot encoding
athens_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
athens_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [athens_onehot.columns[-1]] + list(athens_onehot.columns[:-1])
athens_onehot = athens_onehot[fixed_columns]

print(athens_onehot.shape)
athens_onehot.head()

(6296, 210)


,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Café,Camera Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Deli / Bodega,Dessert Shop,Distillery,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fish Taverna,Food & Drink Shop,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,Nightclub,Optical Shop,Other Great Outdoors,Other Nightlife,Ouzeri,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recording Studio,Recreation Center,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Souvlaki Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Track Stadium,Trail,Tsipouro Restaurant,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Aerides, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Aerides, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Aerides, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [106]:
athens_grouped = athens_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(athens_grouped.shape)
athens_grouped

(63, 210)


,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Café,Camera Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dairy Store,Dance Studio,Deli / Bodega,Dessert Shop,Distillery,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fish Taverna,Food & Drink Shop,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,Nightclub,Optical Shop,Other Great Outdoors,Other Nightlife,Ouzeri,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recording Studio,Recreation Center,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Souvlaki Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Track Stadium,Trail,Tsipouro Restaurant,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Aerides, Athens",0.01,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.010000,0.07000,0.00,0.00,0.000000,0.00,0.00,0.01,0.03,0.000000,0.03,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.000000,0.1300,0.00,0.01,0.00,0.01,0.00,0.06,0.060000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.03000,0.01,0.000000,0.000000,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.010000,0.00000,0.00,0.000000,0.010000,0.00000,0.000000,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.00,0.03,0.00,0.020000,0.00,0.02,0.00,0.000000,0.03,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.01,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01000,0.00,0.00,0.00,0.000000,0.00,0.01,0.000000,0.00,0.00,0.03000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.010000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.0

In [107]:
len(athens_grouped[athens_grouped["Sushi Restaurant"] > 0])

33

#### Create a new DataFrame for Sushi Restaurant data only

In [108]:
athens_sushi = athens_grouped[["Neighborhoods","Sushi Restaurant"]]
athens_sushi.head()

,Neighborhoods,Sushi Restaurant
0,"Aerides, Athens",0.02
1,"Agios Eleftherios, Athens",0.00
2,"Agios Panteleimonas, Athens",0.00
3,Akadimia Platonos,0.00
4,"Akadimia, Athens",0.02


### 7. Cluster Athens Neighborhoods for Sushi Restaurants

In [109]:
# set number of clusters
kclusters = 4

athens_clustering = athens_sushi.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(athens_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return_n_iter=True)


array([2, 1, 1, 1, 2, 1, 2, 1, 1, 2], dtype=int32)

In [110]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
athens_merged = athens_sushi.copy()

# add clustering labels
athens_merged["Cluster Labels"] = kmeans.labels_

athens_merged.head()

,Neighborhoods,Sushi Restaurant,Cluster Labels
0,"Aerides, Athens",0.02,2
1,"Agios Eleftherios, Athens",0.00,1
2,"Agios Panteleimonas, Athens",0.00,1
3,Akadimia Platonos,0.00,1
4,"Akadimia, Athens",0.02,2


In [111]:
athens_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
athens_merged.head()

,Neighborhood,Sushi Restaurant,Cluster Labels
0,"Aerides, Athens",0.02,2
1,"Agios Eleftherios, Athens",0.00,1
2,"Agios Panteleimonas, Athens",0.00,1
3,Akadimia Platonos,0.00,1
4,"Akadimia, Athens",0.02,2


In [112]:
# merge athens_grouped with athens dataframe to add latitude/longitude for each neighborhood
athens_merged = athens_merged.join(athens_df.set_index("Neighborhood"), on="Neighborhood")

print(athens_merged.shape)
athens_merged.head()

(63, 5)


,Neighborhood,Sushi Restaurant,Cluster Labels,Latitude,Longitude
0,"Aerides, Athens",0.02,2,37.976140,23.736400
1,"Agios Eleftherios, Athens",0.00,1,38.019970,23.726270
2,"Agios Panteleimonas, Athens",0.00,1,37.996720,23.727410
3,Akadimia Platonos,0.00,1,37.990950,23.707180
4,"Akadimia, Athens",0.02,2,37.977533,23.737592


In [113]:
# sort the results by Cluster Labels
print(athens_merged.shape)
athens_merged.sort_values(["Cluster Labels"], inplace=True)
athens_merged

(63, 5)


,Neighborhood,Sushi Restaurant,Cluster Labels,Latitude,Longitude
31,Kypriadou,0.01,0,37.993975,23.745041
29,Kountouriotika,0.01,0,37.988563,23.735319
28,Koukaki,0.01,0,37.963720,23.723530
35,"Mets, Athens",0.01,0,37.965220,23.735810
25,Kolokynthou,0.01,0,37.996760,23.706090
36,Monastiraki,0.01,0,37.975220,23.727560
23,"Kallimarmaro, Athens",0.01,0,37.966910,23.740220
22,Kallimarmaro,0.01,0,37.966910,23.740220
39,"Neapoli, Athens",0.01,0,37.986780,23.741480
40,"Neos Kosmos, Athens",0.01,0,37.961420,23.728710


#### Finally, let's visualize the resulting clusters

In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(athens_merged['Latitude'], athens_merged['Longitude'], athens_merged['Neighborhood'], athens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [115]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [116]:
athens_merged.loc[athens_merged['Cluster Labels'] == 0] # Analyze cluster 0

,Neighborhood,Sushi Restaurant,Cluster Labels,Latitude,Longitude
31,Kypriadou,0.01,0,37.993975,23.745041
29,Kountouriotika,0.01,0,37.988563,23.735319
28,Koukaki,0.01,0,37.963720,23.723530
35,"Mets, Athens",0.01,0,37.965220,23.735810
25,Kolokynthou,0.01,0,37.996760,23.706090
36,Monastiraki,0.01,0,37.975220,23.727560
23,"Kallimarmaro, Athens",0.01,0,37.966910,23.740220
22,Kallimarmaro,0.01,0,37.966910,23.740220
39,"Neapoli, Athens",0.01,0,37.986780,23.741480
40,"Neos Kosmos, Athens",0.01,0,37.961420,23.728710


#### Cluster 1

In [117]:
athens_merged.loc[athens_merged['Cluster Labels'] == 1] # Analyze cluster 1

,Neighborhood,Sushi Restaurant,Cluster Labels,Latitude,Longitude
48,"Polygono, Athens",0.0,1,38.002620,23.751310
50,"Profitis Daniil, Athens",0.0,1,37.984610,23.711620
38,Nea Filothei,0.0,1,37.996820,23.763110
51,"Profitis Ilias, Athens",0.0,1,38.028330,23.748320
45,Petralona,0.0,1,37.969480,23.708970
54,"Rouf, Athens",0.0,1,37.976830,23.709890
55,Sepolia,0.0,1,38.004770,23.717940
43,Patisia,0.0,1,38.020190,23.735610
34,Metaxourgeio,0.0,1,37.983040,23.718690
58,Thiseio,0.0,1,37.974810,23.715920


#### Cluster 2

In [118]:
athens_merged.loc[athens_merged['Cluster Labels'] == 2] # Analyze cluster 2

,Neighborhood,Sushi Restaurant,Cluster Labels,Latitude,Longitude
14,"Evangelismos, Athens",0.02,2,37.977150,23.747730
60,Treis Gefyres,0.02,2,37.976140,23.736400
26,Kolonaki,0.02,2,37.977620,23.743590
57,Template:Neighbourhoods in Athens,0.02,2,37.976140,23.736400
4,"Akadimia, Athens",0.02,2,37.977533,23.737592
6,Anafiotika,0.02,2,37.976140,23.736400
17,"Girokomeio, Athens",0.02,2,37.976140,23.736400
52,Psyri,0.02,2,37.976140,23.736400
49,Probonas,0.02,2,37.976140,23.736400
9,"Asyrmatos, Athens",0.02,2,37.931620,23.721151


#### Cluster 3

In [119]:
athens_merged.loc[athens_merged['Cluster Labels'] == 3] # Analyze cluster 3

,Neighborhood,Sushi Restaurant,Cluster Labels,Latitude,Longitude


Cluster 3 is a result of overclustering the data and hence we don't have any information about any neighborhood.

#### Observations

From the clustering process it seems that cluster 2 consists of the most crowded neighborhoods in respect to sushi restaurants so an opening of the restaurant there would most likely suffer from intense competition.
Cluster 0 seems to be populated from many sushi restaurants as well but not as much as cluster 2. Therefore, we conclude that the most attractives neighborhoods for a sushi restaurant establishment is found in cluster 1.
We will continue our analysis for this particular cluster to choose the one neighborhood with the fewest restaurants in general in order to avoid competition from other dining areas.

### 9. Choose the less competitive neighborhood among the ones in cluster 1

In [120]:
athens_cluster_1 = athens_merged.loc[athens_merged['Cluster Labels'] == 1] # Create a new df containing only neighborhoods in Cluster 1
# dropping unnecessary columns
athens_cluster_1.drop(["Sushi Restaurant"], axis = 1, inplace = True) 
athens_cluster_1

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Neighborhood,Cluster Labels,Latitude,Longitude
48,"Polygono, Athens",1,38.002620,23.751310
50,"Profitis Daniil, Athens",1,37.984610,23.711620
38,Nea Filothei,1,37.996820,23.763110
51,"Profitis Ilias, Athens",1,38.028330,23.748320
45,Petralona,1,37.969480,23.708970
54,"Rouf, Athens",1,37.976830,23.709890
55,Sepolia,1,38.004770,23.717940
43,Patisia,1,38.020190,23.735610
34,Metaxourgeio,1,37.983040,23.718690
58,Thiseio,1,37.974810,23.715920


In [121]:
radius = 2000
LIMIT = 100

venues_cluster_1 = []

for lat, long, neighborhood in zip(athens_cluster_1['Latitude'], athens_cluster_1['Longitude'], athens_cluster_1['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results_1 = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results_1:
        venues_cluster_1.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [122]:
# convert the venues list into a new DataFrame
venues_df_cluster_1 = pd.DataFrame(venues_cluster_1)

# define the column names
venues_df_cluster_1.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df_cluster_1.head()

(6296, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Polygono, Athens",38.00262,23.75131,Tourkovounia (Τουρκοβούνια),38.003650,23.755474,Mountain
1,"Polygono, Athens",38.00262,23.75131,Λέσβιον,38.008150,23.753612,Ouzeri
2,"Polygono, Athens",38.00262,23.75131,Cremona,37.998237,23.747005,Cupcake Shop
3,"Polygono, Athens",38.00262,23.75131,Αττικό Άλσος,38.003490,23.760444,Park
4,"Polygono, Athens",38.00262,23.75131,Βιολέτα,38.003015,23.741648,Ouzeri


In [123]:
venues_df_cluster_1.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Agios Eleftherios, Athens",100,100,100,100,100,100
"Agios Panteleimonas, Athens",100,100,100,100,100,100
Akadimia Platonos,100,100,100,100,100,100
"Ampelokipoi, Athens",100,100,100,100,100,100
Ano Petralona,100,100,100,100,100,100
Asteroskopeio,100,100,100,100,100,100
Elaionas,100,100,100,100,100,100
Erythros Stavros,100,100,100,100,100,100
"Gazi, Athens",100,100,100,100,100,100


In [124]:
print('There are {} uniques categories.'.format(len(venues_df_cluster_1['VenueCategory'].unique())))

There are 194 uniques categories.


In [125]:
venues_df_cluster_1['VenueCategory'].unique()[:50]

array(['Mountain', 'Ouzeri', 'Cupcake Shop', 'Park', 'Greek Restaurant',
       'Supermarket', 'Donut Shop', 'Taverna', 'Café', 'Dessert Shop',
       'Czech Restaurant', 'Bakery', 'Indian Restaurant', 'Bookstore',
       'Plaza', 'Bar', 'Breakfast Spot', 'Market', 'Falafel Restaurant',
       'Meze Restaurant', 'Gym', 'Movie Theater', 'Coffee Shop',
       'Cultural Center', 'Souvlaki Shop', 'Italian Restaurant',
       'Theater', 'Pub', 'Asian Restaurant', 'Hookah Bar',
       'Turkish Restaurant', 'Gaming Cafe', 'Cretan Restaurant',
       'Gym / Fitness Center', 'Dance Studio',
       'Middle Eastern Restaurant', 'Mobile Phone Shop',
       'Grilled Meat Restaurant', 'Multiplex', 'Restaurant', 'Hotel Bar',
       'Mediterranean Restaurant', 'Magirio', 'Cocktail Bar',
       'Electronics Store', 'Modern Greek Restaurant', 'Kafenio',
       'Music Venue', 'Nightclub', 'Street Food Gathering'], dtype=object)

In [126]:
# one hot encoding
athens_onehot_cluster_1 = pd.get_dummies(venues_df_cluster_1[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
athens_onehot_cluster_1['Neighborhoods'] = venues_df_cluster_1['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [athens_onehot_cluster_1.columns[-1]] + list(athens_onehot_cluster_1.columns[:-1])
athens_onehot_cluster_1 = athens_onehot_cluster_1[fixed_columns]

print(athens_onehot_cluster_1.shape)
athens_onehot_cluster_1.head()

(2996, 195)


,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Distillery,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fish Taverna,Food & Drink Shop,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,Nightclub,Other Great Outdoors,Other Nightlife,Ouzeri,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recording Studio,Recreation Center,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Souvlaki Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Street Food Gathering,Supermarket,Tailor Shop,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Track Stadium,Trail,Tsipouro Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Polygono, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Polygono, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Polygono, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Polygono, Athens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [127]:
athens_grouped_cluster_1 = athens_onehot_cluster_1.groupby(["Neighborhoods"]).sum().reset_index()

print(athens_grouped_cluster_1.shape)
athens_grouped_cluster_1

(30, 195)


,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Bougatsa Shop,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cretan Restaurant,Cultural Center,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Distillery,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fish Taverna,Food & Drink Shop,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kafenio,Kebab Restaurant,Kids Store,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,Nightclub,Other Great Outdoors,Other Nightlife,Ouzeri,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recording Studio,Recreation Center,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Souvlaki Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Street Food Gathering,Supermarket,Tailor Shop,Tattoo Parlor,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Track Stadium,Trail,Tsipouro Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Agios Eleftherios, Athens",0,0,0,2,0,0,0,0,0,0,7,0,0,1,1,0,0,0,1,0,0,0,2,0,0,0,1,0,13,0,0,0,2,2,0,0,0,0,0,4,0,0,1,0,0,0,7,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,6,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,7,2,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,3,0,0,4,0,2,0,1,0,1,0,0,4,0,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,"Agios Panteleimonas, Athens",0,0,0,0,0,0,0,0,0,0,8,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,6,0,0,0,1,4,0,0,0,0,0,0,1,1,1,0,1,0,7,0,3,0,0,0,0,2,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,5,0,1,0,2,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,3,0,0,0,0,0,1,2,1,0,0,0,2,0,0,4,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,4,0,0,0,0,0,2,0,0,0,1,0,0,6,0,0,0,0,0,0,1,1,1,0,0,2,1,0,0
2,Akadimia Platonos,0,0,1,0,0,1,0,0,0,1,10,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,4,0,1,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,1,9,0,0,0,2,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,1,0,0,0,0,3,0,0,3,1,0,0,2,0,0,0,0,2,0,1,1,0,0,3,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,2,0,0,0,0,1,0,0,2,0,0,3,2,0,0,0,1,2,0,0,0,0,0,1,7,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
3,"Ampelokipoi, Athens",1,0,

In [128]:
cluster1_cols =[x for x in athens_grouped_cluster_1.columns[athens_grouped_cluster_1.columns.str.contains('Restaurant')]]
cluster1_cols

['American Restaurant',
 'Asian Restaurant',
 'Chinese Restaurant',
 'Comfort Food Restaurant',
 'Cretan Restaurant',
 'Czech Restaurant',
 'Ethiopian Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'French Restaurant',
 'German Restaurant',
 'Greek Restaurant',
 'Grilled Meat Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Kebab Restaurant',
 'Latin American Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Meze Restaurant',
 'Middle Eastern Restaurant',
 'Modern European Restaurant',
 'Modern Greek Restaurant',
 'Molecular Gastronomy Restaurant',
 'Restaurant',
 'Scandinavian Restaurant',
 'Seafood Restaurant',
 'Thai Restaurant',
 'Tsipouro Restaurant',
 'Turkish Restaurant',
 'Vegetarian / Vegan Restaurant']

In [129]:
new = athens_grouped_cluster_1[['Neighborhoods','American Restaurant','Asian Restaurant','Chinese Restaurant','Comfort Food Restaurant','Cretan Restaurant','Czech Restaurant',
 'Ethiopian Restaurant','Falafel Restaurant','Fast Food Restaurant','French Restaurant','German Restaurant','Greek Restaurant','Grilled Meat Restaurant','Indian Restaurant',
 'Italian Restaurant','Kebab Restaurant','Latin American Restaurant','Mediterranean Restaurant','Mexican Restaurant','Meze Restaurant','Middle Eastern Restaurant',
 'Modern European Restaurant','Modern Greek Restaurant','Molecular Gastronomy Restaurant','Restaurant','Scandinavian Restaurant','Seafood Restaurant','Thai Restaurant',
 'Tsipouro Restaurant','Turkish Restaurant','Vegetarian / Vegan Restaurant']].copy()
new

,Neighborhoods,American Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cretan Restaurant,Czech Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Grilled Meat Restaurant,Indian Restaurant,Italian Restaurant,Kebab Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Restaurant,Scandinavian Restaurant,Seafood Restaurant,Thai Restaurant,Tsipouro Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,"Agios Eleftherios, Athens",0,0,0,0,0,0,0,0,1,0,0,6,1,0,0,1,0,0,0,7,2,0,0,0,1,0,0,0,0,1,0
1,"Agios Panteleimonas, Athens",0,0,0,0,1,0,0,2,0,0,1,5,0,0,0,0,0,2,0,4,1,0,1,0,1,0,0,0,1,1,1
2,Akadimia Platonos,0,1,0,0,0,0,0,1,0,0,0,9,0,0,0,0,0,3,0,3,1,0,2,0,2,0,1,1,1,0,0
3,"Ampelokipoi, Athens",1,1,0,0,0,1,0,1,0,0,0,3,1,1,1,1,0,1,0,4,0,0,0,0,1,0,0,0,0,0,0
4,Ano Petralona,0,0,0,0,0,0,0,0,0,0,0,8,0,3,0,0,0,2,0,6,0,1,0,0,2,0,0,3,0,0,0
5,Asteroskopeio,0,1,0,0,0,0,0,2,1,0,0,5,1,3,0,0,0,1,0,2,1,0,1,0,1,0,0,1,0,0,0
6,Elaionas,0,0,0,0,1,0,0,0,2,0,0,9,0,0,1,1,0,0,0,1,0,0,0,0,1,1,1,0,1,0,0
7,Erythros Stavros,1,1,0,0,0,1,0,1,0,0,0,4,0,1,1,1,0,1,0,5,0,0,0,0,1,0,0,0,1,0,0
8,"Gazi, Athens",0,1,0,0,0,0,0,2,0,0,0,6,0,3,0,0,0,2,0,3,1,0,1,0,1,0,0,1,0,0,0
9,Goudi,1,1,0,0,1,0,0,1,1,0,0,3,1,0,0,1,0,1,1,1,0,0,0,0,2,0,0,0,1,0,0


In [130]:
new['Total'] = new.sum(axis=1)
final_df = new[['Neighborhoods','Total']]
final_df

,Neighborhoods,Total
0,"Agios Eleftherios, Athens",20
1,"Agios Panteleimonas, Athens",21
2,Akadimia Platonos,25
3,"Ampelokipoi, Athens",17
4,Ano Petralona,25
5,Asteroskopeio,20
6,Elaionas,19
7,Erythros Stavros,19
8,"Gazi, Athens",21
9,Goudi,16


In [131]:
# get the row of min value
final_df.loc[final_df['Total'].idxmin()]

Neighborhoods    Gyzi
Total              11
Name: 11, dtype: object

In [132]:
print('We conclude that the best and less competitive neighborhood to establish a sushi restaurant would be Gyzi.')

We conclude that the best and less competitive neighborhood to establish a sushi restaurant would be Gyzi.


### Final Outcome

It seems that a sushi dining establishment in Gyzi would most likely be profitable and avoid suffering competition from other dining services.